<a href="https://colab.research.google.com/github/ManuBansalS/manuS/blob/main/chapter_appendix-tools-for-deep-learning/jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Run in a notebook cell with ! prefix if needed, or install in your environment beforehand
!pip install -U pip
!pip install numpy pandas nltk spacy scikit-learn gensim matplotlib seaborn joblib
!pip install transformers datasets sentencepiece torch tqdm
# (Optional) spaCy model
!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
INFO: pip is looking at multiple versions of gensim to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 53.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 61.2 MB/s  0:00:05
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 15.4 MB/s  0:00:28
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 68.5 MB/s  0:00:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 56.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━

In [5]:
!pip install --upgrade gensim
!pip install scipy==1.10.1


  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 95.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 93.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 61.2 MB/s  0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.2
    Uninstalling gensim-4.3.2:
      Successfully uninstalled gensim-4.3.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [gensim]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depend

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 118.0 MB/s  0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.9.1 requires scipy>=1.11, but you have scipy 1.10.1 which is incompatible.
scikit-image 0.25.2 requires scipy>=1.11.4, but you have scipy 1.10.1 which is incompatible.
jax 0.5.3 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
jaxlib 0.5.3 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
cvxpy 1.6.7 requires scipy>=1.11.0, but you have scipy 1.10.1 which is incompatible.
arviz 0.22.0 requires scipy>=1.11.0, but you have scipy 1.10.1 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but yo

In [1]:
import os
import random
import numpy as np
import pandas as pd
import re
import string
import nltk
import spacy
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support, roc_auc_score, matthews_corrcoef
import joblib
import matplotlib.pyplot as plt

# Transformers & datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, pipeline
from datasets import Dataset

# Gensim
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel

# SpaCy
nlp = spacy.load("en_core_web_sm")

# NLTK downloads
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
df = pd.read_csv("tweets.csv")  # adjust filename/path
print(df.shape)
df.head(3)


(2107976, 7)


,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0


In [4]:
# Very simple heuristic labeling (not ideal but quick to bootstrap):
# - Tweets containing sad/angry emojis or common negative words -> 'negative'
# - Containing 'thank', 'great', 'good' -> 'positive'
# - else -> 'neutral'
def weak_label(text):
    t = str(text).lower()

    negative_words = [
        'delay', 'late', 'worthless', 'angry', 'bad', 'terrible', 'sucks', 'no service',
        'horrible', 'worst', 'awful', 'disgusting', 'poor', 'useless', 'frustrated', 'annoyed',
        'hate', 'disappointed', 'mess', 'problem', 'broken', 'slow', 'pathetic', 'unacceptable',
        'unhelpful', 'rude', 'careless', 'incompetent', 'fail', 'failure', 'lousy', 'cheap',
        'boring', 'dull', 'mediocre', 'defective', 'flawed', 'inferior', 'ridiculous', 'overpriced',
        'confusing', 'stressful', 'annoying', 'horrid', 'gross', 'subpar', 'painful', 'aggravating',
        'displeased', 'trash', 'junk', 'garbage', 'horrendous', 'clumsy', 'hateful', 'nasty',
        'underwhelming', 'dissatisfied', 'deplorable', 'shoddy', 'unprofessional', 'atrocious',
        'unsatisfactory', 'untrustworthy', 'disaster', 'lacking', 'negligent', 'messy', 'chaotic',
        'overcrowded', 'unreliable', 'noisy', 'pointless', 'disorganized', 'poorly', 'unfair',
        'slow service', 'long wait', 'crashed', 'buggy', 'laggy', 'glitchy', 'corrupt', 'downgrade',
        'locked out', 'waste', 'not worth', 'terribly', 'stupid', 'idiotic', 'misleading',
        'deceptive', 'dangerous', 'unsafe', 'regret', 'frustrating', 'overcomplicated', 'time consuming'
    ]

    positive_words = [
        'thanks', 'thank you', 'great', 'good', 'love', 'awesome', 'excellent', 'amazing',
        'fantastic', 'wonderful', 'perfect', 'superb', 'brilliant', 'outstanding', 'best',
        'satisfied', 'happy', 'pleased', 'delighted', 'impressive', 'fabulous', 'terrific',
        'super', 'cool', 'lovely', 'splendid', 'marvelous', 'exceptional', 'admirable', 'incredible',
        'well done', 'enjoyed', 'glad', 'positive', 'helpful', 'fast', 'speedy', 'quick',
        'efficient', 'polite', 'kind', 'friendly', 'courteous', 'professional', 'reliable',
        'trustworthy', 'brilliant service', 'smooth', 'seamless', 'easy', 'effortless',
        'great job', 'high quality', 'value for money', 'affordable', 'worth it', 'love it',
        'recommend', 'recommended', 'five stars', 'top notch', 'pleasure', 'wonderfully',
        'grateful', 'thankful', 'joyful', 'cheerful', 'fantastically', 'superior', 'commendable',
        'adored', 'cherished', 'favourite', 'favorite', 'magnificent', 'glorious', 'gold standard',
        'best ever', 'best experience', 'positive vibes', 'perfectly', 'beautiful', 'delightful',
        'excellent service', 'user friendly', 'great quality', 'brilliantly', 'painless', 'outperform',
        'stellar', 'flawless', 'outshines', 'remarkable', 'astounding', 'efficiently', 'optimally',
        'swift', 'super smooth', 'exceeded expectations', 'exceptionally good'
    ]

    if any(w in t for w in negative_words):
        return 'negative'
    if any(w in t for w in positive_words):
        return 'positive'
    return 'neutral'


if 'label' not in df.columns:
    df['label'] = df['text'].apply(weak_label)
    print("Weak labels created. Distribution:\n", df['label'].value_counts())


Weak labels created. Distribution:
 label
neutral     1367225
positive     436758
negative     303993
Name: count, dtype: int64


In [5]:
def clean_text(text):
    if pd.isnull(text):
        return ""
    s = str(text)
    s = s.lower()
    s = re.sub(r'http\S+|www\.\S+', '', s)            # remove urls
    s = re.sub(r'@\w+', '', s)                        # mentions
    s = re.sub(r'#', '', s)                           # keep hashtag words, remove #
    s = re.sub(r'\s+', ' ', s).strip()
    s = re.sub(r'[^\x00-\x7f]', '', s)                # remove non-ascii (simple emoji remove)
    # optional: expand common abbreviations (could add a dictionary)
    # lemmatize with spaCy
    doc = nlp(s)
    tokens = [token.lemma_.strip() for token in doc if (not token.is_punct and not token.is_space and token.lemma_ not in STOPWORDS)]
    return " ".join(tokens)

# Apply (can be slow; use .progress_apply with tqdm if large)
from tqdm import tqdm
tqdm.pandas()
df['clean_text'] = df['text'].progress_apply(clean_text)
df[['text','clean_text']].head()


100%|██████████| 2107976/2107976 [6:19:01<00:00, 92.69it/s]


,text,clean_text
0,@115712 I understand. I would like to assist y...,I understand I would like assist would need ge...
1,@sprintcare and how do you propose we do that,propose
2,@sprintcare I have sent several private messag...,I send several private message one respond usual
3,@115712 Please send us a Private Message so th...,please send private message far assist click m...
4,@sprintcare I did.,I


In [6]:
assert 'label' in df.columns, "Need a label column for supervised training"
X = df['clean_text'].values
y = df['label'].values

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.20, stratify=y, random_state=SEED)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=SEED)

print("Sizes: ", len(X_train), len(X_val), len(X_test))


Sizes:  1686380 210798 210798


In [7]:
tfidf = TfidfVectorizer(ngram_range=(1,2), max_features=20000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf = tfidf.transform(X_val)
X_test_tfidf = tfidf.transform(X_test)


In [8]:
lr = LogisticRegression(max_iter=500, random_state=SEED)
params = {'C': [0.01, 0.1, 1, 10]}
clf_lr = GridSearchCV(lr, params, cv=3, scoring='f1_macro', n_jobs=-1)
clf_lr.fit(X_train_tfidf, y_train)
print("Best params:", clf_lr.best_params_)
pred_val = clf_lr.predict(X_val_tfidf)
print("Validation report (LogReg):\n", classification_report(y_val, pred_val))


Best params: {'C': 10}
Validation report (LogReg):
               precision    recall  f1-score   support

    negative       0.98      0.94      0.96     30399
     neutral       0.98      0.99      0.99    136723
    positive       0.99      0.98      0.98     43676

    accuracy                           0.98    210798
   macro avg       0.98      0.97      0.98    210798
weighted avg       0.98      0.98      0.98    210798



In [9]:
svc = LinearSVC(random_state=SEED, max_iter=5000)
params = {'C':[0.01, 0.1, 1, 10]}
clf_svc = GridSearchCV(svc, params, cv=3, scoring='f1_macro', n_jobs=-1)
clf_svc.fit(X_train_tfidf, y_train)
print("Best params:", clf_svc.best_params_)
pred_val_svc = clf_svc.predict(X_val_tfidf)
print("Validation report (SVM):\n", classification_report(y_val, pred_val_svc))


Best params: {'C': 1}
Validation report (SVM):
               precision    recall  f1-score   support

    negative       0.98      0.94      0.96     30399
     neutral       0.98      1.00      0.99    136723
    positive       0.99      0.98      0.98     43676

    accuracy                           0.98    210798
   macro avg       0.98      0.97      0.98    210798
weighted avg       0.98      0.98      0.98    210798



In [10]:
# transform test set already computed
pred_test_lr = clf_lr.predict(X_test_tfidf)
pred_test_svc = clf_svc.predict(X_test_tfidf)

def print_metrics(y_true, y_pred, label="Model"):
    acc = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    print(f"{label} -> Acc: {acc:.4f}, Precision(weighted): {precision:.4f}, Recall(weighted): {recall:.4f}, F1(weighted): {f1:.4f}")

print_metrics(y_test, pred_test_lr, "LogReg")
print_metrics(y_test, pred_test_svc, "SVM")

# confusion matrix (LogReg)
cm = confusion_matrix(y_test, pred_test_lr, labels=np.unique(y))
print("Confusion matrix:\n", cm)


LogReg -> Acc: 0.9824, Precision(weighted): 0.9824, Recall(weighted): 0.9824, F1(weighted): 0.9823
SVM -> Acc: 0.9829, Precision(weighted): 0.9829, Recall(weighted): 0.9829, F1(weighted): 0.9827
Confusion matrix:
 [[ 28429   1635    336]
 [   384 136053    285]
 [    96    974  42606]]


In [11]:
joblib.dump(clf_lr, "logreg_tfidf.joblib")
joblib.dump(clf_svc, "svm_tfidf.joblib")
joblib.dump(tfidf, "tfidf_vectorizer.joblib")


['tfidf_vectorizer.joblib']

In [12]:
# Prepare HuggingFace Dataset objects
df_train = pd.DataFrame({'text': X_train, 'label': y_train})
df_val   = pd.DataFrame({'text': X_val,   'label': y_val})
df_test  = pd.DataFrame({'text': X_test,  'label': y_test})

# Map string labels to integers
label_list = sorted(df['label'].unique())
label2id = {l:i for i,l in enumerate(label_list)}
id2label = {i:l for l,i in label2id.items()}

for d in [df_train, df_val, df_test]:
    d['label'] = d['label'].map(label2id)

hf_train = Dataset.from_pandas(df_train)
hf_val   = Dataset.from_pandas(df_val)
hf_test  = Dataset.from_pandas(df_test)


In [13]:
model_name = "distilbert-base-uncased"  # smaller and faster; swap to 'bert-base-uncased' for full BERT
tokenizer = AutoTokenizer.from_pretrained(model_name)
def tokenize_fn(examples):
    return tokenizer(examples['text'], truncation=True, max_length=128)
hf_train = hf_train.map(tokenize_fn, batched=True, remove_columns=['text'])
hf_val   = hf_val.map(tokenize_fn, batched=True, remove_columns=['text'])
hf_test  = hf_test.map(tokenize_fn, batched=True, remove_columns=['text'])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_list), id2label=id2label, label2id=label2id)

# compute_metrics function
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}

training_args = TrainingArguments(
    output_dir="./bert_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    seed=SEED,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_train,
    eval_dataset=hf_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Map:   0%|          | 0/210798 [00:00<?, ? examples/s]

Map:   0%|          | 0/210798 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'